In [34]:
import os
import random
import numpy as np
import pandas as pd
from pathlib import Path
from copy import deepcopy
from tqdm import tqdm

from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
from sklearn.model_selection import train_test_split

In [35]:
random.seed(42)

In [36]:
def calc_accuracy(y_true, y_pred, metric='mae'):
    if metric == 'mae':
        acc = mean_absolute_error(y_true, y_pred)
    elif metric == 'r2':
        acc = r2_score(y_true, y_pred)
    elif metric == 'rmse':
        acc = root_mean_squared_error(y_true, y_pred)

    return acc

## Consensus methods

In [37]:
from consensus import RandomSearchRegressor, SystematicSearchRegressor, GeneticSearchRegressor, HyperoptSearchRegressor

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge

In [38]:
method_list = [(SystematicSearchRegressor(cons_size=10, metric='r2'), 'Best'),
               #(SystematicSearchRegressor(cons_size=10**3, metric='r2'), 'All'),
               (RandomSearchRegressor(cons_size=2, n_iter=5, metric='r2'), 'Random'),
               (SystematicSearchRegressor(cons_size=2, metric='r2'), 'Systematic'),
               #(GeneticSearchRegressor(cons_size=3, mut_prob=0.5, metric='r2'), 'Genetic'),
               #(HyperoptSearchRegressor(cons_size=3, n_iter=200, metric='rmse'), 'Hyperopt')
               ]

## Consensus comparison
### Input/output data path

In [39]:
bench_folder =  Path("benchmark_model_prediction/chembl").resolve()

### Consensus building

In [40]:
estimators = [
    (LinearRegression(), 'LinearRegression'),
    (RandomForestRegressor(), 'RandomForestRegressor'),
    (MLPRegressor(), 'MLPRegressor'),
    (Ridge(), 'Ridge'),
    (SVR(), 'SVR'),
    (KNeighborsRegressor(), 'KNeighborsRegressor')
]

In [41]:
res_df_val = pd.DataFrame()
res_df_test = pd.DataFrame()
res_df_train_stack = pd.DataFrame()
res_df_test_stack = pd.DataFrame()

In [42]:

for bench_name in tqdm(os.listdir(bench_folder)[:]):

    # load data
    df_val = pd.read_csv(os.path.join(bench_folder, bench_name, f"{bench_name}_val.csv"))
    df_test = pd.read_csv(os.path.join(bench_folder, bench_name, f"{bench_name}_test.csv"))
    df_stack = pd.read_csv(os.path.join(bench_folder, bench_name, f"{bench_name}_stack.csv"))
    
    # remove y_true column prof predictions table
    x_val, y_val = df_val.iloc[:, 1:], df_val.iloc[:, 0]
    x_test, y_test = df_test.iloc[:, 1:], df_test.iloc[:, 0]

    # build consensus
    for method_func, method_name in method_list:
        cons = method_func.run(x_val, y_val)
        y_pred = x_val[cons].mean(axis=1)
        #
        res_df_val.loc[bench_name, method_name] = calc_accuracy(y_val, x_val[cons].mean(axis=1), metric='r2')
        res_df_test.loc[bench_name, method_name] = calc_accuracy(y_test, x_test[cons].mean(axis=1), metric='r2')
        
        # # stacking model
        x_stack, y_stack = df_stack.iloc[:, 1:], df_stack.iloc[:, 0]
        cv = RepeatedKFold(n_splits=4, n_repeats=1, random_state=1)

        for model, name in estimators:

            predictions = cross_val_predict(model, x_test, y_test, cv=cv, n_jobs=-1) 
            res_df_test.loc[bench_name, f"Stacking_test_{model}"] = calc_accuracy(y_test, predictions, metric='r2')
    
    

100%|██████████| 11/11 [00:37<00:00,  3.39s/it]


In [43]:
res_df_test

,Best,Stacking_test_LinearRegression(),Stacking_test_RandomForestRegressor(),Stacking_test_MLPRegressor(),Stacking_test_Ridge(),Stacking_test_SVR(),Stacking_test_KNeighborsRegressor(),Random,Systematic
CHEMBL204,0.744706,-107605.256905,0.774231,-120515.472001,-100043.277926,0.138486,0.639113,0.747799,0.731819
CHEMBL205,0.670250,-441.658204,0.674886,-164.821853,-434.465703,0.511798,0.526749,-16.890398,0.640829
CHEMBL208,0.620633,-0.957822,0.588027,0.121706,0.317017,0.575518,0.541287,0.593201,0.598381
CHEMBL209,0.844755,0.836498,0.879590,-25.107496,0.853913,-0.045120,0.779443,0.803321,0.809199
CHEMBL210,0.653176,-5.131027,0.665907,0.210400,0.319705,0.571407,0.572971,0.646462,0.618914
CHEMBL211,0.709243,0.534837,0.721270,0.585245,0.563839,0.666347,0.629468,0.664537,0.693549
CHEMBL214,0.648332,0.603422,0.662814,0.520925,0.617517,0.625256,0.535745,0.538577,0.643026
CHEMBL216,0.579281,0.078894,0.478045,-0.633319,0.280676,0.506224,0.456679,0.524082,0.537975
CHEMBL217,0.606020,0.554334,0.622749,0.486251,0.566902,0.562010,0.492507,0.585557,0.596750
CHEMBL218,0.535147,0.460442,0.543515,0.453303,0.486400,0.508869,0.475148,0.080892,0.521603


## Meta statistics

In [44]:
from collections import Counter

### 1. Mean accuracy

In [45]:
res_df_val.mean(axis=0).round(2)

Best          0.62
Random       -1.17
Systematic    0.61
dtype: float64

In [46]:
res_df_test.mean(axis=0).round(2)

Best                                         0.60
Stacking_test_LinearRegression()         -9822.68
Stacking_test_RandomForestRegressor()        0.64
Stacking_test_MLPRegressor()            -10973.03
Stacking_test_Ridge()                    -9133.94
Stacking_test_SVR()                          0.46
Stacking_test_KNeighborsRegressor()          0.54
Random                                      -1.06
Systematic                                   0.58
dtype: float64

### 2. Top-N stat

In [47]:
tmp = []
for dataset in res_df_test.index:
    min_i = res_df_test.loc[dataset].argmin()
    max_i = res_df_test.loc[dataset].argmax()
    #
    tmp.append(res_df_test.loc[dataset].index[max_i])
#
Counter(tmp)

Counter({'Stacking_test_RandomForestRegressor()': 8,
         'Best': 2,
         'Stacking_test_Ridge()': 1})

## 3. Pair comapison

In [48]:
# tmp = res_df_test["Best"] - res_df_test["All"]
# tmp = res_df_test["Best"] - res_df_test["Systematic"]
# tmp = res_df_test["All"] - res_df_test["Systematic"]
# tmp = res_df_test["Systematic"] - res_df_test["Genetic"]
# tmp = res_df_test["Systematic"] - res_df_test["Stacking"]
tmp = res_df_test["Genetic"] - res_df_test["Stacking"]


res_pair = Counter(np.where(tmp > 0, 1, 2))
res_pair = {k.item():v for k, v in res_pair.items()}
print(dict(sorted(res_pair.items())))

KeyError: 'Genetic'

In [ ]:
team_1 = ["Res", "All", "Systematic"]
team_2 = ["Genetic", "Stacking"]

res = {1:0, 2:0}
for dataset in res_df_test.index:
    stats = res_df_test.loc[dataset].to_dict()
    alg = max(stats, key=stats.get)
    if alg in team_1:
        res[1] += 1
    else:
        res[2] += 1
res